We start by writing a general class for solving a finite horizon MDP
 

In [1]:
type MDP 
    stateSize  :: Int
    actionSize :: Int
    bellmanUpdate! :: Function
    
    function MDP(c, P)
        (n, m) = size(c)
        
        if length(P) != m
            error("Number of transition matrices does not match the number of actions")
        end
        
        P_concatenated = vcat(P...)
        if size(P_concatenated) != (n*m, n)
            error("Size of transition and reward matrices are inconsistent")
        end
        
        is_square(Pi) = size(Pi) == (n,n)
        is_row_stochastic(Pi) = isapprox(sum(Pi,2), ones(n); atol=100*eps(Float64))
        is_stopping_action(Pi) = Pi - zero(Pi)
        
        for Pi in P
            if !is_square(Pi)
                error("Transition matrix is not a square matrix")
                print(size(Pi))
                print(n)
            elseif !(is_row_stochastic(Pi) || is_stopping_action(Pi))
                error("Transition matrix is not row stochastic")
            end
        end
        
        function update!(v, g, vOld; discount=1)
            Q = c + discount * reshape(P_concatenated * vOld, n, m)
            
            for x=1:n
                g[x], v[x] = 1, Q[x,1]
                for u=2:m
                    if Q[x,u] < v[x]
                        g[x], v[x] = u, Q[x,u]
                    end
                end
            end
        end
        
        new(n, m, update!)
    end
end
            
            

Now we define a function for solving a finite horizon MDP

In [2]:
function finiteHorizon(m::MDP, final_v;
                       discount = 1.0,
                       horizon :: Int = 10)
    
    update!(vUpdated, gUpdated, v) = m.bellmanUpdate!(vUpdated, gUpdated, v; discount=discount)
        
    v = [ zero(final_v) for stage = 1 : horizon ]        
    g = [ zeros(Int,   size(final_v)) for stage = 1 : horizon ]


    update!(v[horizon], g[horizon], final_v)
    for stage in horizon-1: -1 : 1
        update!(v[stage], g[stage], v[stage+1]) 
    end
    return (v,g)    
end


finiteHorizon (generic function with 1 method)

Now, we set the parameters of the model

In [3]:
const rate        = [0.2 0.5 0.8]
const arrivalRate = 0.6

const serviceCost = [0 2 6]
const holdingCost = 2

const R = 1
const M = 8
const A = length(rate)

3

Next, we construct the cost matrix and the transition matrix

In [4]:
P = [ spzeros(Float64, M+1, M+1) for u = 1:A]
c = zeros(Float64, M+1, A)

# Initialize cost matrix
c[1,:] = 0 

for x = 1:(M+1)
    for u = 1:A
        if x == 1 
          c[x,u] = serviceCost[u] 
        else
          c[x,u] = (x-1) * holdingCost + serviceCost[u] - R*rate[u]
        end
    end
end

# Initialize Probability matrix
for x = 2:M
    for u = 1:A
        P[u][x, x-1] = (1 - arrivalRate) * rate[u]
        P[u][x, x]   = (1 - arrivalRate) * (1 - rate[u]) + arrivalRate * rate[u]
        P[u][x, x+1] = arrivalRate * ( 1 - rate[u])
    end
end

for u = 1:A
    P[u][1,1] = (1 - arrivalRate) 
    P[u][1,2] = arrivalRate

    P[u][M+1, M+1] = (1 - rate[u])
    P[u][M+1, M  ] = rate[u]
end

In [5]:
model = MDP(c,P)

MDP(9, 3, update!)

In [6]:
(v,g) = finiteHorizon(model, zeros(M+1); horizon=20)

(Array{Float64,1}[[94.6954, 104.924, 123.537, 149.171, 180.089, 214.201, 248.714, 278.442, 289.822], [88.5741, 98.7763, 117.212, 142.418, 172.58, 205.588, 238.718, 267.033, 277.612], [82.4705, 92.6432, 110.879, 135.611, 164.95, 196.773, 228.431, 255.246, 264.977], [76.3869, 86.5262, 104.539, 128.744, 157.184, 187.739, 217.836, 243.055, 251.899], [70.3259, 80.4275, 98.1879, 121.807, 149.266, 178.471, 206.918, 230.422, 238.377], [64.2907, 74.3493, 91.8239, 114.789, 141.178, 168.95, 195.685, 217.254, 224.499], [58.2851, 68.2944, 85.4436, 107.677, 132.898, 159.152, 184.145, 203.628, 210.37], [52.3137, 62.266, 79.0423, 100.457, 124.402, 149.052, 172.284, 189.746, 195.994], [46.3823, 56.268, 72.6136, 93.1077, 115.666, 138.624, 160.08, 175.664, 181.327], [40.4983, 50.3049, 66.1481, 85.6089, 106.661, 127.842, 147.508, 161.387, 166.562], [34.672, 44.3825, 59.6309, 77.9363, 97.3555, 116.686, 134.53, 146.929, 151.721], [28.9989, 38.4541, 53.0708, 70.0497, 87.7151, 105.159, 121.069, 132.34, 136.81

In [7]:
t = [1,10,15,20]

4-element Array{Int64,1}:
  1
 10
 15
 20

In [8]:
g[t] - 1

4-element Array{Array{Int64,1},1}:
 [0, 2, 2, 2, 2, 2, 2, 2, 1]
 [0, 2, 2, 2, 2, 2, 2, 1, 0]
 [0, 1, 1, 1, 1, 1, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
using Plots
gr()

Plots.GRBackend()

In [10]:
scatter(v[t])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 
 y3 
 
 
 
 
 y4